In [ ]:
import boto3

In [ ]:
with open('hello.py', 'r') as f:
          function_code = f.read()

In [ ]:
print(function_code)

In [ ]:
function_name = 'HelloWorld2'
runtime = 'python3.8'
handler = 'lambda_function.lambda_handler'

In [ ]:
import json
iam_client = boto3.client('iam', region_name='ap-east-1')

In [ ]:
lambda_execution_policy = {
    'Version': '2012-10-17',
    'Statement': [
        {'Effect': 'Allow',
         'Action': [
             'logs:CreateLogGroup',
             'logs:CreateLogStream',
             'logs:PutLogEvents'
         ],
         'Resource': 'arn:aws:logs:*:*:*:*'
        },
    ]}

In [ ]:
role_name = 'LambdaExecutionRole'

In [ ]:
role_response = iam_client.create_role(RoleName=role_name,
                                       AssumeRolePolicyDocument=json.dumps(
                                           {'Version':'2012-10-17',
                                            'Statement': [
                                                {'Effect': 'Allow',
                                                 'Principal': {'Service': 'lambda.amazonaws.com'},
                                                 'Action': 'sts:AssumeRole'
                                                }
                                            ]
                                           }
                                       )
                                      )

In [ ]:
policy_name = 'lambdaExecutionPolicy'

In [ ]:
iam_client.put_role_policy(RoleName=role_name, PolicyName=policy_name, 
                           PolicyDocument=json.dumps(lambda_execution_policy))

In [ ]:
role_arn = role_response['Role']['Arn']

In [ ]:
role_arn

In [ ]:
import zipfile
import io

In [ ]:
lambda_client = boto3.client('lambda', region_name='ap-east-1')

In [ ]:
with io.BytesIO() as deployment_package:
    with zipfile.ZipFile(deployment_package, 'w') as zipf:
        zipf.writestr('lambda_function.py', function_code)
        
    create_function_response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime=runtime,
        Role=role_arn,
        Handler=handler,
        Code={'ZipFile': deployment_package.getvalue()})

In [ ]:
invoke_response = lambda_client.invoke(FunctionName=function_name)

In [ ]:
payload = invoke_response['Payload'].read()

In [ ]:
payload.decode('utf-8')

In [ ]:
lambda_client.delete_function(FunctionName=function_name)

# Lambda Function on AWS -> File uploaded to S3 --> Print the New File (Triggered)

In [ ]:
function_name = 'LambdaTrigger'
runtime = 'python3.8'
handler = 'lambda_function.lambda_trigger'

# ROLE -> Policy -> Rules/Permissions

In [ ]:
# policy_name was CourseLambdaTriggerPoloicy
# role -> CourseLambdaExecutionRoleTrigger

In [ ]:
role_arn = 'arn:aws:iam::339713179183:role/CourseLambdaExecutionRoleTrigger'

In [ ]:
with open('trigger.py', 'r') as f:
    function_code = f.read()

In [ ]:
print(function_code)

In [ ]:
with io.BytesIO() as deployment_package:
    with zipfile.ZipFile(deployment_package, 'w') as zipf:
        zipf.writestr('lambda_function.py', function_code)
    
    create_function_response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime=runtime,
        Role=role_arn,
        Handler=handler,
        Code={'ZipFile': deployment_package.getvalue()})

In [ ]:
s3_client = boto3.client('s3', region_name='ap-east-1')
bucket_name = 'course-test-trigger-sp-new'
s3_client.create_bucket(Bucket=bucket_name,
                       CreateBucketConfiguration={'LocationConstraint': 'ap-east-1'})

In [ ]:
create_function_response['FunctionArn']

In [ ]:
bucket_arn = 'arn:aws:s3:::course-test-trigger-sp-new'

In [ ]:
lambda_client.add_permission(FunctionName=function_name, StatementId='ID1',
                             Action='lambda:InvokeFunction',
                             Principal='s3.amazonaws.com',
                             SourceArn=bucket_arn)

In [ ]:
event_configuration = {
    'LambdaFunctionConfigurations': [
        {'LambdaFunctionArn': create_function_response['FunctionArn'],
         'Events': ['s3:ObjectCreated:*']
        }]
}

In [ ]:
s3_client.put_bucket_notification_configuration(Bucket=bucket_name,
                                                NotificationConfiguration=event_configuration
                                               )

In [ ]:
with open('Test_file.txt', 'w') as file:
          file.write('This is a trigger test')

In [ ]:
s3_client.upload_file(Filename='Test_file.txt', Bucket=bucket_name, Key='Test_file.txt')

In [ ]:
cloudwatch_logs = boto3.client('logs', region_name='ap-east-1')

In [ ]:
response = cloudwatch_logs.describe_log_groups()

In [ ]:
log_streams = cloudwatch_logs.describe_log_streams(logGroupName=f'/aws/lambda/{function_name}',
                                                  orderBy='LastEventTime',
                                                  descending=True)['logStreams']

In [ ]:
cloudwatch_logs.get_log_events(logGroupName=f'/aws/lambda/{function_name}',
                              logStreamName=log_streams[0]['logStreamName'])

In [ ]:
lambda_client.delete_function(FunctionName=function_name)